In [1]:
import numpy as np
import pandas as pd
import re

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
np.random.seed(1)
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import text_to_word_sequence

import matplotlib.pyplot as plt
import gc

import os

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing.text import text_to_word_sequence

stop_words = pd.read_csv('../data/stopwords.csv')['words'].values

def remove_stop_words(text):
    word_tokens = text_to_word_sequence(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return ' '.join(filtered_sentence)

print(stop_words)

['only' 'y' 'by' 'am' 'most' 'me' 'same' 'these' 'so' 'some' 'why' 'down'
 'had' 'd' 'at' 'having' 'those' 'has' 'few' 'theirs' "you've" 'more' 'i'
 'than' 'through' 'be' 'what' 'where' 'myself' 'which' 'doing' 'ours'
 'will' 'in' 'both' 'do' 'it' 'o' 'on' 'yours' 'once' 'ourselves' 'here'
 'about' "it's" 'my' 'for' 'her' 'then' 'after' "should've" 'from' 'each'
 'when' 'does' 'now' 'off' 'don' 'are' 'we' 'itself' 'should' 'his'
 'between' 'our' 'were' 'under' 'other' 'all' 'she' 'won' 'been' "you're"
 'how' 'did' 'yourself' 'they' 'into' 'there' 've' 'such' 't' 's' 'and'
 'over' 'to' 'just' 'was' 'being' 'because' 'if' 'who' 'further' 'the'
 'any' "that'll" 'themselves' 'as' 'again' "you'd" 'until' 'he' 'him'
 'this' 'or' 'of' 'below' 'an' "she's" 'weren' 'm' 'their' 'ma' 'up' 'll'
 'whom' 'hers' 'can' 'you' 'them' 'very' 'a' 'herself' 'before' 'too'
 'himself' 'during' 're' 'out' 'its' 'above' 'own' 'have' 'while'
 'yourselves' 'that' 'with' "you'll" 'is' 'your']


In [3]:
data = pd.concat([pd.read_csv(x) for x in [
    '../data/tweets.csv',
    '../data/word-list.csv',
    '../data/twitter-airline-sentiment.csv',
    '../data/sentiwordnet.csv',
    '../data/reviews.csv',
    '../data/imdb.csv',

]])
len(data)

222914

In [4]:
data['text'] = list(map(remove_stop_words, data['text'].values))
data.head()

,text,pos,neg
0,nbc commissary fountain little things like mak...,0.0,1.0
1,vanilla really good fish tacos,0.0,1.0
2,drinkin listening typical night,0.0,1.0
3,incredible taste hard believe zero contains no...,0.0,1.0
4,doesnt make want vanilla coke zero dont know a...,0.0,1.0


In [5]:
sentences = data['text'].values
corpus = [text_to_word_sequence(y) for y in sentences]
len(corpus)

222914

In [6]:
X_raw_train, X_raw_test, Y_train, Y_test = train_test_split(
    sentences,
    data[['pos', 'neg']].values,
    test_size=0.2, 
    random_state=3945
)

In [7]:
max_sentence_length = 35
vector_size = 100

In [8]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

import os

# word2vec = KeyedVectors.load_word2vec_format('../../GoogleNews-vectors-negative300.bin', binary=True)

word2vec_name = 'lstm-word2vec.bin'

# Create Word2Vec
if os.path.isfile(word2vec_name): 
    print("Loading...")
    word2vec = KeyedVectors.load(word2vec_name)

else:
    print("Computing...")
    word2vec = Word2Vec(sentences=corpus,
                    size=vector_size, 
                    window=10, 
                    negative=20,
                    iter=50,
                    seed=1000,
                    workers=4)
    word2vec.save(word2vec_name)

word2vec = word2vec.wv

gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Computing...


0

In [9]:
def sent2index(corpus):
    gc.collect()
    input_matrix = np.zeros((len(corpus),max_sentence_length))
    
    for i in range(len(corpus)):
        for t, token in enumerate(corpus[i]):
            if t >= max_sentence_length:
                break
            if token not in word2vec.vocab:
                continue
            input_matrix[i, t] = word2vec.vocab.get(token).index
    return input_matrix

In [10]:
X_train = sent2index(X_raw_train)
X_test = sent2index(X_raw_test)

In [11]:
gc.collect()
vocab_len = len(word2vec.vocab) + 1

emb_matrix = np.zeros((vocab_len, vector_size))

for word in word2vec.vocab:
    index = word2vec.vocab.get(word).index
    emb_matrix[index, :] = word2vec[word]
    
print(emb_matrix.shape)

(60229, 100)


In [12]:
gc.collect()
sentence_indices = Input(shape=(max_sentence_length,))
    
embedding_layer = Embedding(vocab_len, vector_size, trainable = False)
embedding_layer.build((None,))
embedding_layer.set_weights([emb_matrix])

embeddings = embedding_layer(sentence_indices)

In [13]:
X = LSTM(128, return_sequences=True)(embeddings)
X = Dropout(0.5)(X)
X = LSTM(128, return_sequences=False)(X)
X = Dropout(0.5)(X)
X = Dense(2, activation=None)(X)
X = Activation('softmax')(X)

model = Model(inputs=[sentence_indices], outputs=X)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 35)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 35, 100)           6022900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 35, 128)           117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
__________

In [14]:
model_file = "keras-model.h5"

def compile_model():
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

def train_model(): 
    model.fit(
        X_train, 
        y=Y_train, 
        batch_size=512, 
        epochs=100, 
        verbose=1, 
        validation_data=(X_test, Y_test))
    
    print("Saving model")
    model.save_weights(model_file)

def load_model():
    model.load_weights(model_file)

In [ ]:
force_train = False

if os.path.isfile(model_file) and not force_train:
    print("Loading model...")
    load_model()
    compile_model()
else:
    print("Training model...")
    compile_model()
    train_model()

Training model...
Train on 178331 samples, validate on 44583 samples
Epoch 1/100
178331/178331 [==============================] - 45s 250us/step - loss: 0.4277 - acc: 0.4685 - val_loss: 0.4268 - val_acc: 0.2969
Epoch 2/100
178331/178331 [==============================] - 43s 241us/step - loss: 0.4238 - acc: 0.4996 - val_loss: 0.4224 - val_acc: 0.4373
Epoch 3/100
178331/178331 [==============================] - 43s 241us/step - loss: 0.4163 - acc: 0.5068 - val_loss: 0.4086 - val_acc: 0.5426
Epoch 4/100
178331/178331 [==============================] - 43s 241us/step - loss: 0.4062 - acc: 0.5017 - val_loss: 0.4016 - val_acc: 0.4526
Epoch 5/100
178331/178331 [==============================] - 43s 242us/step - loss: 0.4006 - acc: 0.5046 - val_loss: 0.3970 - val_acc: 0.5087
Epoch 6/100
178331/178331 [==============================] - 43s 241us/step - loss: 0.3967 - acc: 0.5117 - val_loss: 0.3948 - val_acc: 0.5131
Epoch 7/100
178331/178331 [==============================] - 43s 242us/step - l

Epoch 58/100
178331/178331 [==============================] - 43s 242us/step - loss: 0.3271 - acc: 0.6120 - val_loss: 0.3934 - val_acc: 0.5619
Epoch 59/100
178331/178331 [==============================] - 43s 242us/step - loss: 0.3256 - acc: 0.6143 - val_loss: 0.3961 - val_acc: 0.5641
Epoch 60/100
178331/178331 [==============================] - 44s 246us/step - loss: 0.3241 - acc: 0.6146 - val_loss: 0.4007 - val_acc: 0.5744
Epoch 61/100
178331/178331 [==============================] - 44s 246us/step - loss: 0.3248 - acc: 0.6124 - val_loss: 0.3965 - val_acc: 0.5965
Epoch 62/100
178331/178331 [==============================] - 43s 242us/step - loss: 0.3225 - acc: 0.6142 - val_loss: 0.3998 - val_acc: 0.6129
Epoch 63/100
178331/178331 [==============================] - 43s 243us/step - loss: 0.3228 - acc: 0.6149 - val_loss: 0.3972 - val_acc: 0.5567
Epoch 64/100
178331/178331 [==============================] - 43s 242us/step - loss: 0.3214 - acc: 0.6138 - val_loss: 0.3949 - val_acc: 0.5874

In [ ]:
mess = np.array([
    'this is slow',
    'this is exceptional service',
])
X_test_indices = sent2index(mess)
pred = model.predict(X_test_indices)

output = ''
for i ,m in enumerate(mess):
        output += ('{} {} {}\n'.format('POSITIVE:' if pred[i][0] > 0.5 else 'NEGATIVE:', m, pred[i]))

        
print(output)
        
del mess
del output
del pred
del X_test_indices
gc.collect()

In [ ]:
from IPython.display import clear_output
import os

import urllib, json
import urllib.request

from dotenv import load_dotenv
load_dotenv()

TLGRM_SECRET = os.getenv('TLGRM_SECRET')
TLGRM_MIKE = os.getenv('TLGRM_MIKE')

def telegram_call(method, query = {}):
    try:
        url = 'http://api.telegram.org/bot{}/{}?{}'.format(TLGRM_SECRET, method, urllib.parse.urlencode(query))
        response = urllib.request.urlopen(url)
        return json.loads(response.read().decode("utf-8"))
    except:
        print('Repeating call...')
        return telegram_call(method, query)

def telegram_bot(respond):
    last_offset = 0
    
    while True:
        data = telegram_call('getupdates', {'offset': last_offset})
        for item in data['result']:
            last_offset = item['update_id'] + 1
            if 'message' in item:
                if 'text' in item['message']:
                    text = item['message']['text']
                    chat_id = item['message']['chat']['id']
                    response = respond(text)
                    smd = telegram_call('sendmessage', {
                        'chat_id': chat_id,
                        'text': response
                    })
                    
                    while smd['ok'] == False:
                        smd = telegram_call('sendmessage', {
                            'chat_id': chat_id,
                            'text': response
                        })
                        
                    clear_output()
                    print('FROM: {}\nSAYS: {}\nRESPONSE: {}\n\n'.format(chat_id, text, response))

In [ ]:
from math import floor
gc.collect()

uhms = [
    "uhm", "uhh", "hmm", 
    "hmmm", "oh uh", "oh hmm",
]

sentiment = [
    "😭",
    "☹️",
    "😐",
    "🙂",
    "😍",
]

def respond(text):
    mess = np.array([text])
    X_test_indices = sent2index(mess)
    pred = model.predict(X_test_indices)
    score = pred[0][0]
    sent = sentiment[floor(score * 5)]
    response = '{}: {}'.format(sent, text)

    return response

In [ ]:
telegram_bot(respond)